In [1]:
!pip install great_expectations==1.3.5 pyspark pandas opentelemetry-api opentelemetry-distro opentelemetry-exporter-otlp #opentelemetry-exporter-otlp-proto-grpc  
!pip install opentelemetry-semantic-conventions opentelemetry-util-http wrapt opentelemetry-exporter-otlp-proto-http

In [1]:
# imports. DO NOT REMOVE!
import os, subprocess, time, json, shlex
from quality_sidecar.gx.data_quality_gx import validate_data_quality

In [2]:
DATA_PRODUCT_NAME = 'consuntiviDiProduzione'
EXPECTATIONS_JSON_FILE_PATH = 'resources/csv_v0.1.json'

In [3]:
# OTLP variables. DO NOT REMOVE!
os.environ['OTEL_EXPORTER_OTLP_ENDPOINT'] = 'http://platform-collector:4318'
os.environ['OTEL_EXPORTER_OTLP_PROTOCOL'] = "http/protobuf"
os.environ['OTEL_SERVICE_NAME'] = f'{os.getenv("DATA_PRODUCT_NAME")}-quality_sidecar'

In [4]:
# GX module
validation_results = validate_data_quality(EXPECTATIONS_JSON_FILE_PATH, DATA_PRODUCT_NAME)
print(validation_results)

INFO:root:Reading the JSON configuration file...
INFO:root:Configuring Expectations and running Validations...
INFO:great_expectations.data_context.types.base:Created temporary directory '/tmp/tmp5cgq5iqi' for ephemeral docs site
INFO:root:Created new connector for ('CSV', 'dataSourceSample', 'dataAssetSample')
INFO:root:Expectation instance created: id=None meta={'check_name': 'DataSourceSample_DataAssetSample_VendorId-ToNotBeNull', 'data_product_name': 'consuntiviDiProduzione'} notes=None result_format=<ResultFormat.BASIC: 'BASIC'> description=None catch_exceptions=True rendered_content=None windows=None batch_id=None column='vendor_id' mostly=1 row_condition=None condition_parser=None
Calculating Metrics: 100%|██████████| 8/8 [00:00<00:00, 193.28it/s] 
INFO:root:Reusing connector for ('CSV', 'dataSourceSample', 'dataAssetSample')
INFO:root:Expectation instance created: id=None meta={'check_name': 'DataSourceSample_DataAssetSample_PassengerCount-ToBeBetween0And4', 'data_product_name'

[{'success': True, 'expectation_config': {'type': 'expect_column_values_to_not_be_null', 'kwargs': {'batch_id': 'dataSourceSample-dataAssetSample', 'column': 'vendor_id'}, 'meta': {'check_name': 'DataSourceSample_DataAssetSample_VendorId-ToNotBeNull', 'data_product_name': 'consuntiviDiProduzione'}}, 'result': {'element_count': 10000, 'unexpected_count': 0, 'unexpected_percent': 0.0, 'partial_unexpected_list': [], 'partial_unexpected_counts': [], 'partial_unexpected_index_list': []}, 'meta': {}, 'exception_info': {'raised_exception': False, 'exception_traceback': None, 'exception_message': None}}, {'success': False, 'expectation_config': {'type': 'expect_column_values_to_be_between', 'kwargs': {'batch_id': 'dataSourceSample-dataAssetSample', 'column': 'passenger_count', 'min_value': 0.0, 'max_value': 4.0}, 'meta': {'check_name': 'DataSourceSample_DataAssetSample_PassengerCount-ToBeBetween0And4', 'data_product_name': 'consuntiviDiProduzione'}}, 'result': {'element_count': 10000, 'unexpec

In [5]:
# OTLP module
command = f"opentelemetry-instrument --metrics_exporter otlp,console --logs_exporter otlp,console python -m quality_sidecar.otlp.data_quality_otlp {shlex.quote(json.dumps(validation_results))} \"{DATA_PRODUCT_NAME}\""
result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print(result.stdout.decode())
print(result.stderr.decode())

{
    "resource_metrics": [
        {
            "resource": {
                "attributes": {
                    "telemetry.sdk.language": "python",
                    "telemetry.sdk.name": "opentelemetry",
                    "telemetry.sdk.version": "1.30.0",
                    "service.name": "None-quality_sidecar",
                    "telemetry.auto.version": "0.51b0"
                },
                "schema_url": ""
            },
            "scope_metrics": [
                {
                    "scope": {
                        "name": "__main__",
                        "version": "",
                        "schema_url": "",
                        "attributes": null
                    },
                    "metrics": [
                        {
                            "name": "consuntividiproduzione",
                            "description": "",
                            "unit": "%",
                            "data": {
                                "d